In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark

from pyspark.sql import Row

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
#path

data = '/Users/jerrygeorge/Desktop/finalproj/data/'
model = '/Users/jerrygeorge/Desktop/finalproj/model/'
output = '/Users/jerrygeorge/Desktop/finalproj/output/'

In [4]:
#load the dataset to spark

bus_df = spark.read.parquet(data + 'business.parquet')
user_df = spark.read.parquet(data + 'users.parquet')
rev_df = spark.read.parquet(data + '1.parquet',data +'2.parquet',data +'3.parquet',data +'4.parquet',data +'5.parquet')

In [ ]:
# Convert User id to Integer Type

user_df1 = sqlContext.createDataFrame(user_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("user_id", StringType(), True),StructField("userId", IntegerType(), True)]))



In [6]:
#Join User Id to the table

a = user_df.alias("a")
b = user_df1.alias("b")
    
user_df2 = a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.userId')])

user_df2.select('userId','user_id', 'user_name').show(2)

+------+--------------------+---------+
|userId|             user_id|user_name|
+------+--------------------+---------+
|740469|-0Ji0nOyFe-4yo8BK...|     cubs|
|651612|-1KKYzibGPyUX-Mwk...|   Nickie|
+------+--------------------+---------+
only showing top 2 rows



In [7]:

# Convert Business id to Integer Type

business_df = sqlContext.createDataFrame(bus_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("business_id", StringType(), True),StructField("businessId", IntegerType(), True)]))

business_df.show(2)

+--------------------+----------+
|         business_id|businessId|
+--------------------+----------+
|PfOCPjBrlQAnz__NX...|         0|
|o9eMRCWt5PkpLDE0g...|         1|
+--------------------+----------+
only showing top 2 rows



In [8]:
#Join Business Id to the table

a = bus_df.alias("a")
b = business_df.alias("b")
    
business_df2 = a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.businessId')])

business_df2.select('businessId','business_id', 'business_name').show(2)

+----------+--------------------+--------------------+
|businessId|         business_id|       business_name|
+----------+--------------------+--------------------+
|         0|PfOCPjBrlQAnz__NX...|Brick House Taver...|
|         1|o9eMRCWt5PkpLDE0g...|             Messina|
+----------+--------------------+--------------------+
only showing top 2 rows



In [9]:
# Join User Id, Business Id with the review data

review_df = rev_df.select('user_id', 'business_id', 'stars')


a = review_df.alias("a")
b = user_df1.alias("b")
    
rev_us = a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
                     .select([col('a.'+xx) for xx in a.columns] + [col('b.userId')])

a = rev_us.alias("a")
b = business_df.alias("b")

rev_usb = a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns] + [col('b.businessId')])

rev_usb.show(2)

+--------------------+--------------------+-----+------+----------+
|             user_id|         business_id|stars|userId|businessId|
+--------------------+--------------------+-----+------+----------+
|0y8ORuC2X1i1UF6SG...|--9e1ONYQuAa-CB_R...|    5|598220|     28361|
|9spixZHaqC1JeN1ld...|--9e1ONYQuAa-CB_R...|    2|203254|     28361|
+--------------------+--------------------+-----+------+----------+
only showing top 2 rows



In [10]:
#Rating Matirx

rat_df = rev_us.select('userId', 'businessId', rev_usb.stars.cast('float').alias('rating'))
rat_df.show(2)
print(' Rating matrx no. of rows :', rat_df.count())
rat_df.printSchema()

+------+----------+------+
|userId|businessId|rating|
+------+----------+------+
|598220|     28361|   5.0|
|203254|     28361|   2.0|
+------+----------+------+
only showing top 2 rows

 Rating matrx no. of rows : 3221419
root
 |-- userId: integer (nullable = true)
 |-- businessId: integer (nullable = true)
 |-- rating: float (nullable = true)



In [12]:
(train, test) = rating_df.randomSplit([0.8, 0.2], seed=123)


In [ ]:

als = ALS(userCol="userId", itemCol="businessId", ratingCol="rating", coldStartStrategy="drop")

pg = ParamGridBuilder().addGrid(
    als.rank,
    [10,15,16,18,20],
).addGrid(
    als.maxIter,
    [10,15,16,18,20],
).build()

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
)

cv = CrossValidator(estimator=als, estimatorParamMaps=pg, evaluator=evaluator, numFolds=10, seed=123)
cvm = cv.fit(train)

#Rmse
predi = cvm.bestModel.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predi)
print("Root-mean-square error = " + str(rmse))

modl = cvm.bestModel

modl.rank

(best_model
    ._java_obj     
    .parent()      
    .getMaxIter()) 


# Root-mean-square error is 1.4


In [13]:
#final AlS
als1 = ALS(rank=20, maxIter=20, regParam=0.3, userCol="userId", itemCol="businessId", ratingCol="rating", \
               coldStartStrategy="drop", seed=123)
mdl3 = als1.fit(train)

pred2 = als1.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(alsb_predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 1.362570061835842


In [16]:

#Top 20 restaurant for Users
Urecom = als1_model.recommendForAllUsers(20)

In [ ]:
#Recommendation sample
a = Urecom.alias("a")
b = user_df1.alias("b")
    
allrecom = a.join(b, col("a.userId") == col("b.userId"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.user_id')])

allrecom.cache()   
allrecom.show(1, truncate=False)

In [ ]:
# sShow Recommendation

u_id = 'ZWD8UH1T7QXQr0Eq-mcWYg'

rec =  sqlContext.createDataFrame(Urecom.filter(col('user_id') == u_id).rdd.flatMap(lambda p: p[1]))
rec.show()

In [19]:
#Recommended Restaurant Details
a = business_new_df.alias("a")
b = rec.alias("b")

ucolab = a.join(b, col("a.businessId") == col("b.businessId"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns] + [col('b.rating')])
    
ucolab.select('business_id', 'business_name', 'rating', 'categories').toPandas()

,business_id,business_name,rating,categories
0,Mglgm8rfJxZ0LLgN23YGOg,Kimchi Korea,5.207201,"[Restaurants, Food, Shopping, Korean]"
1,YwDZUnYeWyNFgBelSdE-Fw,Hatch,5.068435,"[Food, Restaurants, Cafes, Coffee & Tea]"
2,otsjAjxf0PNQ99xcmuj_LA,Sushi Making For the Soul,4.950624,"[Japanese, Restaurants]"
3,v_OLzcpFA7vgVp30vxv2uQ,Silver Spoon,4.900938,"[American (New), Canadian (New), Restaurants]"
4,Pyf26htaH5h_GoKbuHTwGA,Banh Mi Ba Le,4.904104,"[Vietnamese, Canadian (New), Restaurants]"
5,2vA_jqF4XKPgWQ_h4pxRlQ,Sheli's,4.961460,"[Restaurants, Sandwiches, Burgers, Chicken Wings]"
6,n_LSGXDDApdFS9EC2WGUGw,Salerno,4.929758,"[Pizza, Meat Shops, Sandwiches, Restaurants, F..."
7,ixllnoaJuCh6gIy7Weqzmg,Porchetta Roll,5.011046,"[Sandwiches, Restaurants]"
8,B8ROl31v2PAi3Zw5oYc4Fw,Can-Cey Restaurant,5.163110,"[Restaurants, Diners]"
9,0gEPvzMBpVv6M75cdITSGg,Sushi Café Myoung Dong Soon To Fu Korea Restau...,5.117366,"[Restaurants, Korean]"


In [1]:

#Display Map
def smap(df):
    
    mp = folium.Map(location=[38, -75], zoom_start=10)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[r.latitude, r.longitude], 
                    popup = html.escape(r["business_name"]) + '<br>' + 'Stars: ' + str(r.stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='red')).add_to(mp)

    return mp

In [21]:
#get Recommendations

def colab(u_id):
    

    rec =  sqlContext.createDataFrame(URec.filter(col('user_id') == u_id).rdd.flatMap(lambda p: p[1]))

    a = rec.alias("a")
    b = business_df2.alias("b")
    
    return a.join(b, col("a.businessId") == col("b.businessId"), 'inner') \
             .select([col('b.business_id'), col('a.rating'), col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')]) \
             .orderBy("rating", ascending = False)

In [22]:
#Recommendations
u_id = 'ZWD8UH1T7QXQr0Eq-mcWYg'

colab(u_id).toPandas()

,business_id,rating,business_name,categories,stars,review_count,latitude,longitude
0,Mglgm8rfJxZ0LLgN23YGOg,5.207201,Kimchi Korea,"[Restaurants, Food, Shopping, Korean]",4.5,4,43.590970,-79.598077
1,B8ROl31v2PAi3Zw5oYc4Fw,5.163110,Can-Cey Restaurant,"[Restaurants, Diners]",4.5,3,43.737478,-79.244436
2,0gEPvzMBpVv6M75cdITSGg,5.117366,Sushi Café Myoung Dong Soon To Fu Korea Restau...,"[Restaurants, Korean]",5.0,6,43.664527,-79.414051
3,YwDZUnYeWyNFgBelSdE-Fw,5.068435,Hatch,"[Food, Restaurants, Cafes, Coffee & Tea]",4.5,3,43.845812,-79.361602
4,ixllnoaJuCh6gIy7Weqzmg,5.011046,Porchetta Roll,"[Sandwiches, Restaurants]",5.0,9,43.624263,-79.605990
5,2vA_jqF4XKPgWQ_h4pxRlQ,4.961460,Sheli's,"[Restaurants, Sandwiches, Burgers, Chicken Wings]",4.0,3,43.798950,-79.446329
6,otsjAjxf0PNQ99xcmuj_LA,4.950624,Sushi Making For the Soul,"[Japanese, Restaurants]",4.5,3,43.656233,-79.392319
7,n_LSGXDDApdFS9EC2WGUGw,4.929758,Salerno,"[Pizza, Meat Shops, Sandwiches, Restaurants, F...",5.0,4,45.570776,-73.639838
8,Pyf26htaH5h_GoKbuHTwGA,4.904104,Banh Mi Ba Le,"[Vietnamese, Canadian (New), Restaurants]",4.5,11,43.604056,-79.590778
9,v_OLzcpFA7vgVp30vxv2uQ,4.900938,Silver Spoon,"[American (New), Canadian (New), Restaurants]",5.0,4,43.650883,-79.450832


In [ ]:
#map

import folium
import html
u_id = 'ZWD8UH1T7QXQr0Eq-mcWYg'
smap(getCollabRecom(u_id))